__node__

https://github.com/nodejs/docker-node/issues/533

According to the README, it looks like I should be able to just create a docker-compose file with the following contents to get an application running:
```
version: "2"
services:
  node:
    image: "node:8"
    environment:
      - NODE_ENV=production
    volumes:
      - ./:/usr/src/app
    expose:
      - "8080"
```
The note underneath implies that simply performing a `docker-compose up` will start a container in which the dependencies will be installed via an `npm install` and that the application will be started by a `npm start` command. However, `docker-compose up` just starts a container, executes the node command in the root directory and instantly stops the container since nothing is being executed. This is normal, since the Dockerfile for the `node:8` image only contains `CMD[ "node" ]` and no working directory.

If you want to run straight from the image you need to at least add a `working_dir: /usr/src/app` and an entrypoint to the `docker-compose.yml`. In the entrypoint you'll need to execute a bash script in which you first perform an `npm install` followed by an `npm start`.

Npm does not allow running as root by default for security reasons. When you run npm as root (this is the default user in Docker build) and install a global package, npm installs and executes binaries as user `nobody` instead, who doesn't have any permissions.

You can avoid this by adding the `--unsafe-perm` flag:

```
RUN npm install --global --unsafe-perm exp
```
or by setting the global user explicitly to root:

```
RUN npm --global config set user root && \
    npm --global install exp
```
